In [12]:
import pandas as pd
from sentence_transformers import SentenceTransformer

In [31]:
model = SentenceTransformer("all-MiniLM-L6-v2")

def vector_similarity_metric(model_answers: pd.DataFrame, ground_truth: pd.DataFrame):  
    #Find score using vector similarities
    score_matrix = model_answers.apply(
        lambda row_model: ground_truth.apply(
            lambda row_truth: 
                model.similarity(model.encode(row_truth["sentence"]), model.encode(row_model["sentence"])).item() 
                if row_model["comedian"] == row_truth["comedian"] 
                else None,
        axis=1),
    axis=1) \
        .melt(ignore_index=False) \
        .dropna() \
        .reset_index()

    # Find the best score by taking the maximum value
    best_match = score_matrix.loc[score_matrix.groupby("index")["value"].idxmax()].reset_index(drop=True)

    # Add in the ground truths that were not matched
    missing_gt = set(ground_truth.index) - set(best_match["variable"].unique())
    missing_gt = pd.DataFrame({"index": None, "variable": list(missing_gt), "value": 0})
    
    result = best_match.append(missing_gt).groupby("variable").mean().rename(columns={"value": "score"})

    # Calculate penalty for over generation 
    over_generation_penalty = len(model_answers) - len(ground_truth)
    penalty_factor = max(over_generation_penalty, 0)  
    
    # # Add in the comedians and group by the mean for each comedian
    final_result = result.merge(ground_truth[['comedian']], left_index=True, right_index=True).reset_index(drop=True)
    comedian_scores = final_result.groupby('comedian')['score'].mean().reset_index() 
    
    # Apply penalty to the final score, if penalty goes under 0, score is set to 0
    comedian_scores['score'] -= penalty_factor * 0.1 
    comedian_scores['score'] = comedian_scores['score'].apply(lambda x: max(x, 0))
    comedian_scores['score'] *= 100
    
    return comedian_scores

In [32]:
ground_truth = pd.read_csv('/home/ada/humor/data/stand_up_dataset/standup_data.csv')
gemma = pd.read_csv('/home/ada/humor/data/stand_up_dataset/gemma_answers.csv')

gemma_metric = vector_similarity_metric(gemma, ground_truth)

In [34]:
gemma_metric.mean()

/tmp/ipykernel_2323956/2897354298.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  gemma_metric.mean()


score    30.00032
dtype: float64

In [35]:
import sys
import pandas as pd
sys.path.append("..")
from scipy.stats import spearmanr
from humor.bipartite_metric import bipartite_metric

model = pd.read_csv('/home/ada/humor/data/stand_up_dataset/gemma_answers.csv')

gm = bipartite_metric(model, ground_truth)
merged_df = pd.merge(gm, gemma_metric, on='comedian', suffixes=('_df1', '_df2'))
correlation, p_value = spearmanr(merged_df['score_df1'], merged_df['score_df2'])
print("Correlation: ", correlation)
print("p_value:", p_value)

Correlation:  0.8182805429864253
p_value: 2.2627778718074984e-13
